In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold, LeaveOneOut
from scipy.spatial.distance import pdist, squareform
from matplotlib.patches import Ellipse
from scipy.stats import chi2
import matplotlib.colors as mcolors


# Function to evaluate models and save results
def evaluate_models(directory_path):
    csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    results = []

    for csv_file in csv_files:
        file_path = os.path.join(directory_path, csv_file)
        polymers = pd.read_csv(file_path)
        polymers_data = polymers[polymers["test"] == 1]
        y = polymers_data["sample"]
        x = polymers_data.drop(["test", "sample"], axis=1)

        # Determine the maximum number of components for LDA
        n_classes = len(y.unique())
        n_features = x.shape[1]
        max_components = min(n_classes - 1, n_features)

        # Perform Stratified K-Fold Cross-Validation
        stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=4)
        test_scores = []

        for train_index, test_index in stratified_kfold.split(x, y):
            X_train, X_test = x.iloc[train_index], x.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            lda = LinearDiscriminantAnalysis(n_components=max_components).fit(X_train, y_train)
            score = lda.score(X_test, y_test)
            test_scores.append(score)

        overall_accuracy = np.mean(test_scores)
        
        # Perform Leave-One-Out Cross-Validation (Jackknife)
        loo = LeaveOneOut()
        loo_scores = []

        for train_index, test_index in loo.split(x):
            X_train, X_test = x.iloc[train_index], x.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            lda = LinearDiscriminantAnalysis(n_components=max_components).fit(X_train, y_train)
            score = lda.score(X_test, y_test)
            loo_scores.append(score)

        average_score = np.mean(loo_scores)
        
        # Calculate centroids for each class
        x_scores = lda.transform(x)
        x_scores_df = pd.DataFrame(x_scores, columns=[f'Component {i+1}' for i in range(x_scores.shape[1])])
        x_scores_df['sample'] = y.values
        centroids = x_scores_df.groupby('sample').mean()

        # Calculate Euclidean distances between centroids
        centroid_distances = pdist(centroids.values, metric='euclidean')
        mean_centroid_distance = np.mean(centroid_distances)
        max_centroid_distance = np.max(centroid_distances)
        min_centroid_distance = np.min(centroid_distances)
        
        # Find the classes corresponding to the minimum and maximum distances
        distance_matrix = squareform(centroid_distances)
        min_dist_indices = np.unravel_index(np.argmin(distance_matrix + np.eye(len(centroids)) * np.max(distance_matrix)), distance_matrix.shape)
        max_dist_indices = np.unravel_index(np.argmax(distance_matrix), distance_matrix.shape)
        
        min_distance_classes = (centroids.index[min_dist_indices[0]], centroids.index[min_dist_indices[1]])
        max_distance_classes = (centroids.index[max_dist_indices[0]], centroids.index[max_dist_indices[1]])

        # Compute variance statistics
        variances = lda.explained_variance_ratio_
        max_variance = np.max(variances)
        min_variance = np.min(variances)
        avg_variance = np.mean(variances)

        # Compute the variance of centroids
        centroid_variance = np.var(centroids.values, axis=0)
        total_centroid_variance = np.mean(centroid_variance)

        # Store results
        results.append({
            'File': csv_file,
            'Stratified K-Fold Accuracy': overall_accuracy * 100,
            'LOO CV Accuracy': average_score * 100,
            'Mean Centroid Distance': mean_centroid_distance,
        })
        
        # Save heatmap for the Euclidean distances with scale bar from 0 to 100
        plt.figure(figsize=(10, 8))
        # Optional: Visualize the distance matrix as a heatmap with custom colormap
        cmap = mcolors.LinearSegmentedColormap.from_list("custom_cmap", ["#80BE81", "#FBEA92", "#D96E6C"])

        sns.heatmap(squareform(centroid_distances), cmap=cmap, xticklabels=centroids.index, yticklabels=centroids.index, vmin=0, vmax=200)
        #plt.title(f"Heatmap of Euclidean Distances_n_component=2_sample: {csv_file}")
        #plt.savefig(f"Heatmap_Euclidean_Distances_{csv_file}_n_component=2.png", dpi=300)
        plt.close()

    # Save results to a CSV file
    results_df = pd.DataFrame(results)
    results_df.to_csv("all_first.csv", index=False)
    print("\nResults saved to 'LDA_Evaluation_Results_Homo.csv'.")

# Run evaluation on the directory
directory_path = "data/all_first"  # Specify your directory path here
evaluate_models(directory_path)



Results saved to 'LDA_Evaluation_Results_Homo.csv'.


In [ ]:
#2D polt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import LeaveOneOut, StratifiedKFold
from matplotlib.patches import Ellipse
from scipy.stats import chi2


A="data/AB_fromPCA/LDA_AB_4sensors_Normalized_fromPCA(2)_9.csv"
def plot_lda_with_ellipse(scale_bool):
    polymers = pd.read_csv(A) #440/470, 440/510
    polymers_train = polymers[polymers["test"]==1]
    y_train_dummies = polymers_train["sample"]
    x_train = polymers_train.drop(["test", "sample"], axis=1)

    lda = LinearDiscriminantAnalysis(n_components=2)
    x_scores = lda.fit(x_train, y_train_dummies).transform(x_train)

    fig, ax = plt.subplots(1, 1, figsize=(8, 8))

    cmap = {'AX-B120':"magenta", 'AX-B121': "#009193", 'AX-B135':"peru", 'AX-B136':"green", 'AX-B137':"orange", 'AX-B138':"blue", 'AX-B140':"tomato", 'AX-B141':"purple", 'AX-B139':"darkblue", 
             'AX-B164': "#44840C", 'AX-B166': "#945200", 'AX-B167': "#0096FF", 'AX-B168': "#716DF6", 'AX-B169': "#FADFB6", 'AX-B170': "#F5B455"
           , 'AX-B171': "#F4A636", 'AX-B194': "#F8CA87", 'AX-B195': "#DC0D23", 'AX-B196': "#83800F"}

    # 目盛の調整
    plt.rcParams['axes.linewidth'] = 2
    ax.tick_params(width=2)
    ax.tick_params(axis='both', which='both', direction='in', length=6)
    
    ax.set_xlim(-80, 60)
    #ax.set_xticks([-70, -60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60])
    ax.set_ylim(-20, 25)
    #ax.set_yticks([-35, -30, -25, -20, -15, -10, -5, 0, 5, 10, 15, 20, 25])  

    ax.set_xlabel('Score(1)')
    ax.set_ylabel('Score(2)')
    ax.set_title('LDA Score Plot')
    
    

    for i, (t1, t2) in enumerate(x_scores):
        color = cmap[polymers_train["sample"].iloc[i]]
        plt.scatter(t1, t2, c='None', edgecolors=color, marker="o", label='None', linewidth=1,  s=50)

        if scale_bool:
            class_data = x_scores[y_train_dummies == y_train_dummies.iloc[i]]
            class_covariance = np.cov(class_data.T)
            n = class_data.shape[0]
            pooled_covariance = class_covariance * (n - 1) / (n - 2)
            mean = np.mean(class_data, axis=0)
            eigenvalues, eigenvectors = np.linalg.eigh(pooled_covariance)
            order = eigenvalues.argsort()[::-1]
            eigenvalues, eigenvectors = eigenvalues[order], eigenvectors[:, order]
            angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))
            width, height = 2 * np.sqrt(eigenvalues * chi2.ppf(0.95, 2))
            ell = Ellipse(xy=(mean[0], mean[1]), width=width, height=height, angle=angle,
                          edgecolor=color, linestyle='-', linewidth=2, fill=False)
            ax.add_patch(ell)
            
    #plt.savefig("LDA_4sensors_ratio-All-H5A(-).png", dpi=600)
    plt#show()
    
    explained_variance_ratio = lda.explained_variance_ratio_
    np.set_printoptions(precision=4)
    print("Explained Variance Ratio (LDA):", explained_variance_ratio)


def lda_loocv():
    polymers = pd.read_csv(A) #440/470, 440/510
    polymers_data = polymers[polymers["test"] == 1]
    y = polymers_data["sample"]
    x = polymers_data.drop((["test", "sample"]), axis=1)

    loo = LeaveOneOut()

    misclassified = []
    actual_classes = []
    predicted_classes = []

    for train_index, test_index in loo.split(x):
        X_train, X_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        lda = LinearDiscriminantAnalysis(n_components=2).fit(X_train, y_train)
        prediction = lda.predict(X_test)

        if prediction != y_test.values[0]:
            misclassified.append((y_test.values[0], prediction[0]))

        actual_classes.append(y_test.values[0])
        predicted_classes.append(prediction[0])

    for actual, predicted in misclassified:
        print(f"Actual Class: {actual}, Predicted Class: {predicted}")

    accuracy = np.mean(np.array(actual_classes) == np.array(predicted_classes))
    print('Leave-One-Out Cross-Validation Accuracy (LOOCV): {:.2f}%'.format(accuracy * 100))


def lda_kfold():
    polymers = pd.read_csv(A) #440/470, 440/510
    polymers_data = polymers[polymers["test"] == 1]
    y = polymers_data["sample"]
    x = polymers_data.drop((["test", "sample"]), axis=1)

    stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
    test_scores = []

    for train_index, test_index in stratified_kfold.split(x, y):
        X_train, X_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        lda = LinearDiscriminantAnalysis(n_components=2).fit(X_train, y_train)
        score = lda.score(X_test, y_test)
        test_scores.append(score)
        print('Test set score: {}'.format(score))

    overall_accuracy = np.mean(test_scores)
    print('Overall Test Set Accuracy: {:.2f}%'.format(overall_accuracy * 100))


def main():
    lda_loocv()
    lda_kfold()
    plot_lda_with_ellipse(scale_bool=1)


if __name__ == "__main__":
    main()

In [ ]:
#3d polt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from itertools import combinations

A = "data/ab_fromPCA/LDA_AB_4sensors_Normalized_fromPCA(2).csv"

def plot_lda_with_ellipse_3d(component1=1, component2=2, component3=3, x_range=None, y_range=None, z_range=None):
    polymers = pd.read_csv(A)
    polymers_train = polymers[polymers["test"] == 1]
    y_train_dummies = polymers_train["sample"]
    x_train = polymers_train.drop(["test", "sample"], axis=1)

    lda = LinearDiscriminantAnalysis(n_components=3)
    x_scores = lda.fit(x_train, y_train_dummies).transform(x_train)

    explained_variance_ratio = lda.explained_variance_ratio_

    fig = go.Figure()

    cmap = {'AX-B120': "magenta", 'AX-B135': "peru", 'AX-B136': "green", 'AX-B137': "orange", 'AX-B138': "blue", 
            'AX-B140': "tomato", 'AX-B141': "purple", 'AX-B139': "darkblue", 'AX-B121': "#009193", 
            'AX-B164': "#44840C", 'AX-B166': "#945200", 'AX-B167': "#0096FF", 'AX-B168': "#716DF6", 
            'AX-B169': "#FADFB6", 'AX-B170': "#F5B455", 'AX-B171': "#F4A636", 'AX-B194': "#F8CA87", 
            'AX-B195': "#DC0D23", 'AX-B196': "#83800F"}

    selected_scores = x_scores[:, [component1-1, component2-1, component3-1]]

    for sample in polymers_train["sample"].unique():
        mask = polymers_train["sample"] == sample
        points = selected_scores[mask]

        # プロット同士をすべてペアにして線を描く
        for p1, p2 in combinations(points, 2):
            fig.add_trace(go.Scatter3d(
                x=[p1[0], p2[0]],
                y=[p1[1], p2[1]],
                z=[p1[2], p2[2]],
                mode='lines',
                line=dict(color=cmap[sample], width=1),
                showlegend=False  # すべての線に対して凡例を表示しない
            ))

        # 各クラスの点を表示
        fig.add_trace(go.Scatter3d(
            x=points[:, 0],
            y=points[:, 1],
            z=points[:, 2],
            mode='markers',
            marker=dict(size=1, color=cmap[sample]),
            name=sample
        ))

    fig.update_layout(
        scene=dict(
            xaxis=dict(title=f'LDA Component {component1} ({explained_variance_ratio[component1-1]*100:.2f}%)',
                       range=x_range),  # x軸の範囲を設定
            yaxis=dict(title=f'LDA Component {component2} ({explained_variance_ratio[component2-1]*100:.2f}%)',
                       range=y_range),  # y軸の範囲を設定
            zaxis=dict(title=f'LDA Component {component3} ({explained_variance_ratio[component3-1]*100:.2f}%)',
                       range=z_range),  # z軸の範囲を設定
            aspectmode='cube'  # 正方形の軸を指定
        ),
        title=f'LDA 3D Plot: Components {component1}, {component2}, {component3}',
        width=1200,  # グラフの幅
        height=800   # グラフの高さ
    )

    fig.show()

    np.set_printoptions(precision=4)
    print("Explained Variance Ratio (LDA):", explained_variance_ratio)

# 軸範囲を指定する例
plot_lda_with_ellipse_3d(component1=1, component2=2, component3=3, x_range=[-60, 60], y_range=[-40, 20], z_range=[-40, 30])
